In [1]:
%reload_ext autoreload
%autoreload 1
%aimport src.dataset_utils
%aimport src.display_utils
%aimport src.batch_utils
%aimport src.classifier_basic
%aimport src.classifier_ml

import src.dataset_utils as dataset_utils
import src.batch_utils as batch_utils
import src.classifier_basic as classifier_basic
import src.classifier_ml as classifier_ml

## Dataset

[High-Resolution Fundus (HRF) Image Database](https://www5.cs.fau.de/research/data/fundus-images/) - maintained by FAU Erlangen-Nürnberg

The HRF dataset contains 45 entries of high-resolution fundus images. When arranged lexicographically, the first 5 entries are used for showcase purposes, while the remaining 40 entries are used for training and testing purposes.

**Note:** Due to licensing issues, none of the visualizations are displayed in this repository. To visualize the results, please run the code locally.

In [2]:
dataset_dir = 'hrf'

dataset_utils.prepare_dataset(dataset_dir)
dataset_manager = dataset_utils.DatasetManager(dataset_dir)

entry_ids_showcase = list(range(0, 5))
entry_ids_train_test = list(range(5, dataset_manager.dataset_size))

In [3]:
classifier_basic.evaluate_collection(dataset_manager, entry_ids_showcase)

01_dr
| accuracy           : 0.96 |
| sensitivity        : 0.73 |
| specificity        : 0.97 |
| g-mean(sens, spec) : 0.84 |
\___________________________/


01_g
| accuracy           : 0.96 |
| sensitivity        : 0.71 |
| specificity        : 0.98 |
| g-mean(sens, spec) : 0.84 |
\___________________________/


01_h
| accuracy           : 0.95 |
| sensitivity        : 0.62 |
| specificity        : 0.99 |
| g-mean(sens, spec) : 0.78 |
\___________________________/


02_dr
| accuracy           : 0.96 |
| sensitivity        : 0.68 |
| specificity        : 0.98 |
| g-mean(sens, spec) : 0.82 |
\___________________________/


02_g
| accuracy           : 0.96 |
| sensitivity        : 0.70 |
| specificity        : 0.98 |
| g-mean(sens, spec) : 0.83 |
\___________________________/




## Batch processing

The images from the HRF dataset are processed in full resolution, which is 3504x2336 pixels.

Due to RAM constraints, processing these images in full resolution is not feasible on most machines, so the images are divided into smaller batches. Batches are saved locally and then processed one by one.

Current setup requires 32 GB of RAM and 80 GB of disk space.

In [4]:
batch_manager = batch_utils.BatchManager('batches')

batch_info = batch_utils.BatchInfo(patch_side=21,
                                   patch_step_train_test=5,
                                   patch_step_showcase=1,
                                   entry_ids_train_test=entry_ids_train_test,
                                   entry_ids_showcase=entry_ids_showcase,
                                   bpi_test=2,
                                   bpi_showcase=4,
                                   train_size=0.7,
                                   under_sampling_strat=0.3)

batch_utils.batch(dataset_manager, batch_manager, batch_info)

Batching showcase: 100%|██████████| 5/5 [02:25<00:00, 29.06s/entry]


In [5]:
model = classifier_ml.create_model(max_iter=500,
                                   learning_rate=0.1,
                                   min_samples_leaf=40,
                                   max_leaf_nodes=121,
                                   max_depth=None,
                                   l2_regularization=0.2,
                                   scoring='loss')

classifier_ml.train_and_test(batch_manager, batch_info, model)
classifier_ml.evaluate_collection(dataset_manager, batch_manager, batch_info, model)

Training...


Testing: 100%|██████████| 40/40 [03:57<00:00,  5.94s/entry]


| accuracy           : 0.96 |
| sensitivity        : 0.82 |
| specificity        : 0.97 |
| g-mean(sens, spec) : 0.89 |
\___________________________/


01_dr
| accuracy           : 0.95 |
| sensitivity        : 0.81 |
| specificity        : 0.96 |
| g-mean(sens, spec) : 0.88 |
\___________________________/


01_g
| accuracy           : 0.96 |
| sensitivity        : 0.80 |
| specificity        : 0.97 |
| g-mean(sens, spec) : 0.88 |
\___________________________/


01_h
| accuracy           : 0.96 |
| sensitivity        : 0.81 |
| specificity        : 0.98 |
| g-mean(sens, spec) : 0.89 |
\___________________________/


02_dr
| accuracy           : 0.94 |
| sensitivity        : 0.83 |
| specificity        : 0.95 |
| g-mean(sens, spec) : 0.89 |
\___________________________/


02_g
| accuracy           : 0.96 |
| sensitivity        : 0.82 |
| specificity        : 0.97 |
| g-mean(sens, spec) : 0.89 |
\___________________________/


